# Accuracy Assessment of Water Observations from Space (WOfS) Product in Africa


The cleaned validation samples from the previous step `02b_Convert_Institution_to_AEZ.ipynb` are ingested here to create confusion mattrices for each agro-ecological zone, and one for the entire continent.


**Input data** : `<AEZ>_wofs_ls_validation_points.csv>`

**Output_data** : `<AEZ>_confusion_matrix.csv`

Last modified: 03/02/2022


### Load packages
Import Python packages that are used for the analysis.

In [1]:
%matplotlib inline

import sys
import os
import rasterio
import xarray
import glob
import numpy as np
import pandas as pd
import seaborn as sn
import geopandas as gpd
import matplotlib.pyplot as plt
import scipy, scipy.ndimage
import warnings
warnings.filterwarnings("ignore") #this will suppress the warnings for multiple UTM zones in your AOI 

from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import plot_confusion_matrix, f1_score  
from deafrica_tools.plotting import map_shapefile,display_map, rgb
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.datahandling import wofs_fuser, mostcommon_crs,load_ard,deepcopy
from deafrica_tools.dask import create_local_dask_cluster

## Load the datasets
Ground truth points 

In [2]:
#Read the ground truth data 
#For each AEZ
file_path = ('../02_Validation_results/WOfS_Assessment/wofs_ls/')
validation_files = [i for i in glob.glob(os.path.join(file_path, '*.{}'.format('csv')))]
validation_files

['../02_Validation_results/WOfS_Assessment/wofs_ls/Northern_wofs_ls_validation_points.csv',
 '../02_Validation_results/WOfS_Assessment/wofs_ls/Central_wofs_ls_validation_points.csv',
 '../02_Validation_results/WOfS_Assessment/wofs_ls/Sahel_wofs_ls_validation_points.csv',
 '../02_Validation_results/WOfS_Assessment/wofs_ls/Western_wofs_ls_validation_points.csv',
 '../02_Validation_results/WOfS_Assessment/wofs_ls/Indian_ocean_wofs_ls_validation_points.csv',
 '../02_Validation_results/WOfS_Assessment/wofs_ls/Southern_wofs_ls_validation_points.csv',
 '../02_Validation_results/WOfS_Assessment/wofs_ls/Eastern_wofs_ls_validation_points.csv']

## Create a continental validation dataset

In [3]:
continental = pd.concat([pd.read_csv(f) for f in validation_files])

## Function for creating confusion matrix

In [4]:
def create_confusion_matrix(df, aez):

    # create a confusion matrix
    confusion_matrix = pd.crosstab(
        df["ACTUAL"],
        df["PREDICTION"],
        rownames=["ACTUAL"],
        colnames=["PREDICTION"],
        margins=True,
    )
    
    #producer's accuracy
    confusion_matrix["Producer's"] = [
        confusion_matrix.loc[0][0] / confusion_matrix.loc[0]["All"] * 100,
        confusion_matrix.loc[1][1] / confusion_matrix.loc[1]["All"] * 100,
        np.nan,
    ]
    
    #user's acc
    users_accuracy = pd.Series(
        [
            confusion_matrix[0][0] / confusion_matrix[0]["All"] * 100,
            confusion_matrix[1][1] / confusion_matrix[1]["All"] * 100,
        ]
    ).rename("User's")

    confusion_matrix = confusion_matrix.append( 
        users_accuracy
    )
    
    #overall acc
    confusion_matrix.loc["User's", "Producer's"] = (
        (confusion_matrix[0][0] + confusion_matrix[1][1])
        / confusion_matrix["All"]["All"]
        * 100
    )
    df["PREDICTION"] = df["PREDICTION"].astype(str).astype(int)
    
    #fscore
    fscore = pd.Series(
        [
            (
                2
                * (
                    confusion_matrix.loc["User's"][0]
                    * confusion_matrix.loc[0]["Producer's"]
                )
                / (
                    confusion_matrix.loc["User's"][0]
                    + confusion_matrix.loc[0]["Producer's"]
                )
            )
            / 100,
            f1_score(df["ACTUAL"], df["PREDICTION"]),
        ]
    ).rename("F-score")
    
    #tidy confusion matrix
    confusion_matrix = confusion_matrix.append(fscore)
    confusion_matrix = confusion_matrix.round(decimals=2)
    confusion_matrix = confusion_matrix.rename(
        columns={
            "0": "NoWater",
            "1": "Water",
            0: "NoWater",
            1: "Water",
            "All": "Total",
        },
        index={"0": "NoWater", "1": "Water", 0: "NoWater", 1: "Water", "All": "Total"},
    )
    
    #remove the nonsensical values in the table
    confusion_matrix.loc["User's", 'Total'] = '--'
    confusion_matrix.loc['Total', "Producer's"] = '--'
    confusion_matrix.loc["F-score", 'Total'] = '--'
    confusion_matrix.loc["F-score", "Producer's"] = '--'
    
    print('\n')
    print('n samples for', aez,':', len(df))
    print(confusion_matrix)
    # saving out the confusion matrix
    confusion_matrix.to_csv('../02_Validation_results/WOfS_Assessment/wofs_ls/ConfusionMatrix/'+aez+'_confusion_matrix.csv')

## AEZ confusion matrices

In [5]:
for v in validation_files:
    df = pd.read_csv(v, delimiter=",")
    aez = v[49:-30]
    create_confusion_matrix(df, aez)




n samples for Northern : 1125
         NoWater   Water   Total Producer's
ACTUAL                                     
NoWater   393.00   71.00   464.0       84.7
Water     113.00  548.00   661.0       82.9
Total     506.00  619.00  1125.0         --
User's     77.67   88.53      --      83.64
F-score     0.81    0.86      --         --


n samples for Central : 606
         NoWater   Water  Total Producer's
ACTUAL                                    
NoWater    59.00   15.00   74.0      79.73
Water     113.00  419.00  532.0      78.76
Total     172.00  434.00  606.0         --
User's     34.30   96.54     --      78.88
F-score     0.48    0.87     --         --


n samples for Sahel : 1224
         NoWater   Water   Total Producer's
ACTUAL                                     
NoWater   390.00  120.00   510.0      76.47
Water      80.00  634.00   714.0       88.8
Total     470.00  754.00  1224.0         --
User's     82.98   84.08      --      83.66
F-score     0.80    0.86      --    

## Continental confusion matrix

In [6]:
df = pd.concat([pd.read_csv(f) for f in validation_files])

create_confusion_matrix(df, 'Continental')



n samples for Continental : 11117
         NoWater    Water    Total Producer's
ACTUAL                                       
NoWater  2713.00   377.00   3090.0       87.8
Water    1616.00  6411.00   8027.0      79.87
Total    4329.00  6788.00  11117.0         --
User's     62.67    94.45       --      82.07
F-score     0.73     0.87       --         --


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** January 2020

**Compatible datacube version:** 